### 배포 - npm build 후에 배포

In [1]:
import paramiko

import os
def get_file_list(path):
    return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

hostname = "maxy.iptime.org"
# hostname = "192.168.10.177"
username = "songs"
password = "thdghqkr74"
port = 8022
remote_path = "/mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front"
# SSH 클라이언트 객체 생성
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [2]:

def check_remote_dir(sftp, remote_path):
    """원격 경로가 없으면 계층적으로 생성"""
    dirs = remote_path.strip('/').split('/')
    path = ''
    for d in dirs:
        path += '/' + d
        try:
            sftp.stat(path)  # 존재 여부 확인
        except FileNotFoundError:
            sftp.mkdir(path)
            

def FileUpload(sftp,SourceFile,TargetFile):
    try:
        with sftp.open(TargetFile, "wb") as remote_file, open(SourceFile, "rb") as local_file:
            chunk_size = 1024 * 1024  # 32KB씩 다운로드
            downloaded_size = 0
            while True:
                data = local_file.read(chunk_size)
                if not data:
                    break
                remote_file.write(data)
                downloaded_size += len(data)
                print(f"업로드중: {SourceFile} -> {downloaded_size}")
        print(f"완료: {TargetFile}")
    except Exception as e:
        print(f"FileUpload 오류 발생: {SourceFile}-{TargetFile}-{e}")
        
        
def FolderUpdate(sftp,SourceFolder,TargetFolder):
    try:
        # flist = get_file_list(f"{SourceFolder}")
        flist = os.listdir(f"{SourceFolder}")
        for file in flist:
            if file == "." or file == "..":
                continue
            if os.path.isdir(f"{SourceFolder}/{file}"):
                check_remote_dir(sftp,f"{TargetFolder}/{file}")
                FolderUpdate(sftp,f"{SourceFolder}/{file}",f"{TargetFolder}/{file}")
                continue
            SourceFile = f"{SourceFolder}/{file}"
            TargetFile = f"{TargetFolder}/{file}"
            FileUpload(sftp,SourceFile,TargetFile)
    except Exception as e:
        print(f"FolderUpdate 오류 발생: {SourceFolder}-{TargetFolder}-{e}")

In [3]:
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh_client.connect(hostname, port, username, password)
    TargetPath = f"{remote_path}"
    SourcePath = f"."
    # SFTP 클라이언트 생성
    sftp = ssh_client.open_sftp()
    FileUploadDir = ["app","public","types","public","components","vendor","i18n"]
    for folder in FileUploadDir:
        check_remote_dir(sftp,f"{TargetPath}/{folder}")
        FolderUpdate(sftp,f"{SourcePath}/{folder}",f"{TargetPath}/{folder}")    
    UploadFiles = [
        (f"{SourcePath}/package-lock.json",f"{TargetPath}/package-lock.json"),
        (f"{SourcePath}/package.json",f"{TargetPath}/package.json"),
        (f"{SourcePath}/settings.ts",f"{TargetPath}/settings.ts"),
        (f"{SourcePath}/tsconfig.json",f"{TargetPath}/tsconfig.json"),
    ]
    for SourceFile,TargetFile in UploadFiles:
        FileUpload(sftp,SourceFile,TargetFile)
    sftp.close()
except Exception as e:
    print(f"오류 발생: {e}")
finally:
    ssh_client.close()

업로드중: ./app/favicon.ico -> 25931
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/favicon.ico
업로드중: ./app/.DS_Store -> 6148
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/.DS_Store
업로드중: ./app/layout.tsx -> 1346
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/layout.tsx
업로드중: ./app/lib/tippy.ts -> 3285
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/lib/tippy.ts
업로드중: ./app/api/Widget/ResourceUsage.ts -> 3887
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/api/Widget/ResourceUsage.ts
업로드중: ./app/api/Widget/Logmeter.ts -> 8771
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/api/Widget/Logmeter.ts
업로드중: ./app/api/Widget/Accessibility.ts -> 2373
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/api/Widget/Accessibility.ts
업로드중: ./app/api/Widget/Favorites.ts -> 2882
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_front/app/api/Widget/Favorites.ts
업로드중: ./app/api/Widget/PageView.ts -> 3624
완료

In [4]:
# flist = get_file_list(f".next")
# os.listdir(path=".next")